In [66]:
NOTEBOOK_NAME = "e010_make_train_retrieve_same_lrg_cd"

In [44]:
import os
from glob import glob

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import japanize_matplotlib

from contextlib import contextmanager
from time import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
import shutil
from tqdm.auto import tqdm

%matplotlib inline


# ref: Kaggleコード遺産 https://qiita.com/kaggle_grandmaster-arai-san/items/d59b2fb7142ec7e270a5 
class Timer:
    def __init__(self, logger=None, format_str="{:.3f}[s]", prefix=None, suffix=None, sep=" "):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)


def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    
# 再現性確保!
seed_everything(33)

In [45]:
INPUT_DIR = "../data"
OUTPUT_DIR = f"../saved_data/{NOTEBOOK_NAME}"

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [46]:
# 学習用のログデータと正解ラベル
train_log = pd.read_csv(os.path.join(INPUT_DIR, "train_log.csv"))
train_session = pd.read_csv(os.path.join(INPUT_DIR, "train_label.csv"))

# 宿のデータ
yado = pd.read_csv(os.path.join(INPUT_DIR, "yado.csv"))

# テスト期間のログデータ
test_log = pd.read_csv(os.path.join(INPUT_DIR, "test_log.csv"))
test_session = pd.read_csv(os.path.join(INPUT_DIR, "test_session.csv"))

sample_submission = pd.read_csv(os.path.join(INPUT_DIR, "sample_submission.csv"))

In [47]:
whole_log_df = pd.concat([train_log, test_log], ignore_index=True)

In [48]:
def add_negative_sample(
    log_df: pd.DataFrame, session_df: pd.DataFrame, yado: pd.DataFrame, N: int
):
    # yad_noとlrg_cdの辞書
    yad_no_lrg_cd_dict = dict(zip(yado["yad_no"], yado["lrg_cd"]))
    # session_idとyad_noの辞書
    session_id_yad_no_dict = (
        log_df.groupby("session_id")["yad_no"].apply(list).to_dict()
    )
    # lrg_cdとyad_noの辞書
    lrg_cd_yad_no_dict = yado.groupby("lrg_cd")["yad_no"].apply(list).to_dict()

    # logに存在したyad_noを追加
    session_df["logged_yad_no"] = session_df["session_id"].map(session_id_yad_no_dict)

    # logの最後のyad_noを追加
    session_df["last_yad_no"] = session_df["logged_yad_no"].apply(lambda x: x[-1])
    # logの最後のyad_noに対するlrg_cdを追加
    session_df["last_lrg_cd"] = session_df["last_yad_no"].map(yad_no_lrg_cd_dict)
    session_df["last_yad_no"] = session_df["logged_yad_no"].apply(lambda x: x[-1])

    # 最後のlogのデータから、同じlrg_cdのyad_noを追加
    session_df["last_yad_no_lrg_cd"] = session_df["last_yad_no"].map(yad_no_lrg_cd_dict)

    session_df["same_lrg_cd_yad_no"] = session_df["last_yad_no_lrg_cd"].map(
        lrg_cd_yad_no_dict
    )

    # same_lrg_cd_yad_noの中から、logged_yad_noには存在しないyad_noをnegative_sampleとして10件追加
    negative_samples = []
    for logged_yad_no, same_lrg_cd_yad_no in tqdm(
        zip(session_df["logged_yad_no"], session_df["same_lrg_cd_yad_no"]),
        total=len(session_df),
    ):
        same_lrg_cd_yad_no_wo_logged_yad_no = list(
            set(same_lrg_cd_yad_no) - set(logged_yad_no)
        )
        negative_yad_no_10 = random.sample(
            same_lrg_cd_yad_no_wo_logged_yad_no,
            min(N, len(same_lrg_cd_yad_no_wo_logged_yad_no)),
        )
        negative_samples.append(negative_yad_no_10)

    session_df["negative_sample"] = negative_samples

    negative_data = (
        session_df[["session_id", "negative_sample"]]
        .explode("negative_sample")
        .rename(columns={"negative_sample": "yad_no"})
    )

    negative_data = negative_data.sort_values(["session_id", "yad_no"])

    return negative_data


# add_negative_sample(train_log, train_session, yado, N=10)
# add_negative_sample(test_log, test_session, yado, N=100_000)

In [49]:
def create_session_train_yad_df(
    log_df: pd.DataFrame, label_df: pd.DataFrame, yado: pd.DataFrame
):
    # 負例を追加
    negative_data = add_negative_sample(log_df, label_df, yado, N=10)

    # ランダムに付け加えたもの以外・同一ログに出現する宿を候補にいれる
    no_dup_train_log = log_df[["session_id", "yad_no"]].drop_duplicates()
    out_df = pd.concat([no_dup_train_log, negative_data], ignore_index=True)

    # 正解のデータを追加
    out_df = pd.concat([label_df, out_df], ignore_index=True)

    # 正解ラベルに含まれているレコードの index を配列で取得して
    target_index = pd.merge(
        out_df.reset_index(), label_df, on=["session_id", "yad_no"], how="inner"
    )["index"].values

    # 正解Indexに含まれている場合 1 / そうでないと 0 のラベルを作成
    out_df["reserve"] = out_df.index.isin(target_index).astype(int)

    # 重複を省く
    out_df = out_df.drop_duplicates(subset=["session_id", "yad_no"], keep="first")

    # 見た目を揃えるために session / yad の順番でソートをします
    out_df = out_df.sort_values(["session_id", "yad_no"]).reset_index(drop=True)

    # 必要な列に限定
    out_df = out_df[["session_id", "yad_no", "reserve"]]

    return out_df

In [50]:
train_session_yad = create_session_train_yad_df(
    log_df=train_log, label_df=train_session, yado=yado
)

  0%|          | 0/288698 [00:00<?, ?it/s]

In [51]:
# TODO: ここから
# testデータの一番最後に存在するログは必ず正解ではないため、除外する
def remove_last_yad_id(test_log: pd.DataFrame, test_session_yad: pd.DataFrame):
    # セッション中一番最後の宿の組を作成
    last_yad_df = test_log.groupby("session_id").tail(1)[["session_id", "yad_no"]]

    # 最後であることがわかるようにラベル is_last を付与
    last_yad_df["is_last"] = 1

    # 引数の session - yad の組み合わせとマージ
    merged = test_session_yad.merge(
        last_yad_df, on=["session_id", "yad_no"], how="left"
    )

    # is_last **ではない** (i.e. is_last is null) データのみに絞る
    idx_use = merged["is_last"].isnull()
    test_session_yad = test_session_yad[idx_use].reset_index(drop=True)

    return test_session_yad


remove_last_yad_id(test_log=test_log, test_session_yad=test_session_yad)

,session_id,yad_no
0,00001149e9c73985425197104712478c,337
1,00001149e9c73985425197104712478c,360
2,00001149e9c73985425197104712478c,930
3,00001149e9c73985425197104712478c,1063
4,00001149e9c73985425197104712478c,1286
...,...,...
14670521,ffffe984aafd6127ce8e43e3ca40c79d,13582
14670522,ffffe984aafd6127ce8e43e3ca40c79d,13593
14670523,ffffe984aafd6127ce8e43e3ca40c79d,13695
14670524,ffffe984aafd6127ce8e43e3ca40c79d,13735


In [52]:
def create_session_test_yad_df(
    log_df: pd.DataFrame, session_df: pd.DataFrame, yado: pd.DataFrame
):
    negative_data = add_negative_sample(
        log_df, session_df, yado, N=100_000_000
    )  # 無駄に大きい値を入れて、常に全件取るようにする

    # ランダムに付け加えたもの以外・同一ログに出現する宿を候補にいれる
    no_dup_train_log = log_df[["session_id", "yad_no"]].drop_duplicates()
    out_df = pd.concat([no_dup_train_log, negative_data], ignore_index=True)

    # testデータの一番最後に存在するログは必ず正解ではないため、除外する
    out_df = remove_last_yad_id(test_log=log_df, test_session_yad=out_df)

    # 重複を省く
    out_df = out_df.drop_duplicates(subset=["session_id", "yad_no"], keep="first")

    # 見た目を揃えるために session / yad の順番でソートをします
    out_df = out_df.sort_values(["session_id", "yad_no"]).reset_index(drop=True)

    # 必要な列に限定
    out_df = out_df[["session_id", "yad_no"]]

    return out_df

In [53]:
test_session_yad = create_session_test_yad_df(
    log_df=test_log, session_df=test_session, yado=yado
)

  0%|          | 0/174700 [00:00<?, ?it/s]

TypeError: 'tuple' object is not callable

In [54]:
print(train_session_yad.shape)
print(test_session_yad.shape)

(3454134, 3)
(14670526, 2)


In [64]:
train_session_yad[train_session_yad["session_id"] == "000104bdffaaad1a1e0a9ebacf585f33"]

,session_id,yad_no,reserve
48,000104bdffaaad1a1e0a9ebacf585f33,96,1
49,000104bdffaaad1a1e0a9ebacf585f33,655,0
50,000104bdffaaad1a1e0a9ebacf585f33,809,0
51,000104bdffaaad1a1e0a9ebacf585f33,898,0
52,000104bdffaaad1a1e0a9ebacf585f33,1533,0
53,000104bdffaaad1a1e0a9ebacf585f33,3253,0
54,000104bdffaaad1a1e0a9ebacf585f33,3596,0
55,000104bdffaaad1a1e0a9ebacf585f33,4743,0
56,000104bdffaaad1a1e0a9ebacf585f33,5490,0
57,000104bdffaaad1a1e0a9ebacf585f33,5551,0


In [67]:
train_session_yad.to_pickle(f"{OUTPUT_DIR}/{NOTEBOOK_NAME}_merged_train.pkl")
test_session_yad.to_pickle(f"{OUTPUT_DIR}/{NOTEBOOK_NAME}_merged_test.pkl")